In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON
from collections import defaultdict
import textwrap

In [12]:
# Endpoint
sparql = SPARQLWrapper("https://shared.semantics.cancer.gov/sparql")

In [13]:
# Query for Alzheimer's (C2866)
# Core properties:
# - ncit:P97: DEFINITION
# - ncit:P90: FULL_SYN (Synonyms)
# - rdfs:label: Preferred Name
# Additional interoperability properties:
# - ncit:P106: Semantic_Type
# - ncit:P207: UMLS_CUI
query_string = """
PREFIX ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>

SELECT DISTINCT ?prop_label ?prop_value
FROM <http://ncicb.nci.nih.gov/xml/owl/EVS/ThesaurusInf.rdf>
WHERE {
  VALUES ?subject { ncit:C2866 }
  ?subject ?prop ?prop_value .
  ?prop rdfs:label ?prop_label .
  ?prop a owl:AnnotationProperty .
}
ORDER BY ?prop_label
"""

In [14]:
sparql.setQuery(query_string)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

In [15]:
# Prepare containers for the different properties
core_props = {
    "Preferred_Name": None,
    "DEFINITION": None,
    "ALT_DEFINITION": None,
    "FULL_SYN": []
}
interop_props = {
    "Semantic_Type": None,
    "UMLS_CUI": None,
}
other_props = defaultdict(list)  # for any other property to be shown optionally

# Populate containers from the results
for b in results["results"]["bindings"]:
    label = b["prop_label"]["value"]
    value = b["prop_value"]["value"]
    # CORE
    if label == "Preferred_Name":
        core_props["Preferred_Name"] = value
    elif label == "DEFINITION":
        core_props["DEFINITION"] = value
    elif label == "ALT_DEFINITION":
        core_props["ALT_DEFINITION"] = value
    elif label == "FULL_SYN":
        core_props["FULL_SYN"].append(value)
    # INTEROP
    elif label == "Semantic_Type":
        interop_props["Semantic_Type"] = value
    elif label == "UMLS_CUI":
        interop_props["UMLS_CUI"] = value
    else:
        other_props[label].append(value)

print("=" * 120)
print("CORE PROPERTIES")
print("=" * 120)

pref = core_props["Preferred_Name"] if core_props["Preferred_Name"] else "N/A"
definition = core_props["DEFINITION"] if core_props["DEFINITION"] else "N/A"
alts = []
if core_props["ALT_DEFINITION"]:
    alts.append(core_props["ALT_DEFINITION"])
alldefs = [definition] + alts if definition != "N/A" else alts
altdef_str = " / ".join(filter(None, alldefs))

synonyms = core_props["FULL_SYN"] if core_props["FULL_SYN"] else ["N/A"]

print(f"Preferred Name:\n- {pref}\n")
print("Synonyms:")
for syn in synonyms:
    print(f"- {syn}")
print()

print("Definition(s):")
for chunk in textwrap.wrap(altdef_str, width=110):
    print(chunk)

print("\n" + "=" * 120)
print("INTEROPERABILITY PROPERTIES")
print("=" * 120)
print(f"Semantic Type: {interop_props['Semantic_Type'] if interop_props['Semantic_Type'] else 'N/A'}")
print(f"UMLS CUI: {interop_props['UMLS_CUI'] if interop_props['UMLS_CUI'] else 'N/A'}")



CORE PROPERTIES
Preferred Name:
- Alzheimer's Disease

Synonyms:
- Alzheimer's Disease
- Alzheimer disease
- Alzheimer's Dementia
- Alzheimer's disease
- Alzheimer's disease, unspecified
- Alzheimer Disease
- Alzheimer dementia

Definition(s):
A progressive, neurodegenerative disease characterized by loss of function and death of nerve cells in several
areas of the brain leading to loss of cognitive function such as memory and language. / A brain disorder that
usually starts in late middle age or old age and gets worse over time. Symptoms include loss of memory,
confusion, difficulty thinking, and changes in language, behavior, and personality.

INTEROPERABILITY PROPERTIES
Semantic Type: Mental or Behavioral Dysfunction
UMLS CUI: C0002395
